**Trabalho prático #1**

Nome: Filipe Rodrigues Batista de Oliveira - 2018055091

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import sim

id_cliente = sim.simxStart("127.0.0.1", 19999, True, True, 5000, 5)

NOME_ROBO = "Quadcopter"
OBJS = ["Joao", "casa", "helicoptero", "arvore", "moto"]

if id_cliente != -1:
    rv, handle_robo = sim.simxGetObjectHandle(id_cliente, NOME_ROBO, sim.simx_opmode_oneshot_wait)
    
    for obj in OBJS:
        rv, pos = sim.simxGetObjectPosition(id_cliente, obj, handle_robo, sim.simx_opmode_oneshot_wait)
        print(pos)
    
    sim.simxFinish(id_cliente)
    
else:
    print("Não foi possível conectar ao Coppelia Sim!")

Não foi possível conectar ao Coppelia Sim!
